# Setup class, func

In [ ]:
# !nvidia-smi

In [1]:
# !git clone https://github.com/Tuan-Lee-23/Thesis-Chatbot.git
# %cd Thesis-Chatbot

Cloning into 'Thesis-Chatbot'...
remote: Enumerating objects: 440, done.
remote: Counting objects: 100% (125/125), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 440 (delta 43), reused 121 (delta 39), pack-reused 315
Receiving objects: 100% (440/440), 431.60 MiB | 19.64 MiB/s, done.
Resolving deltas: 100% (45/45), done.
Checking out files: 100% (347/347), done.
/content/Thesis-Chatbot


In [2]:
# %cd Thesis-Chatbot/
# path prefix cho kaggle

# prefix = "./Thesis-Chatbot/"
prefix = ""

In [3]:
# !pip install -q -r Thesis-Chatbot/requirements.txt

In [4]:
params = {
    # CustomDataset config-------------
    'seqlen': 256, # nên chỉnh cực thấp (64) để debug lỗi từ dataloader, memory,...
    'context_len': 233, 
    'response_len': 23, #seqlen = context_len + respone_len
    'img_features_json': 'data/k60_img_features.json', #path cho file img_features json

    # Model config-------------------
    #tên huggingface pretrained model
    'pretrained_model': "microsoft/DialoGPT-small",
    #hidden size của phiên bản pretrained hiện tại
    'hidden_size': 768,
    #img features size: kích thước của feature ảnh (efficient deepcluster là 768, mặc định gốc là 1792)
    'img_features_size': 768,
    # dropout prob for all fully connected layers in: embeddings, encoder, and pooler
    'embed_dropout': 0,
    # dropout prob for the attention.
    'attention_dropout': 0,
    # dropout prob for residual connection
    'resid_dropout': 0,

    # Training setup-------------------
    'random_seed': 234, # seed, con số may mắn =))
    'num_samples': 30000, # số dòng dữ liệu để train
    'num_cpus': 0, # số nhân cpu của máy, để xử lí song song dataloader
    'epochs': 14,
    'batch_size': 16,
    'lr': 3e-4,
    'max_lr': 1e-3,
    'weight_decay': 0.001,
    'use_gradient_checkpointing': False, # True thì giảm memory nhưng giảm luôn training speed
    'gradient_accumulation': 8,
    'val_check_interval': 0.5,  # tần suất chạy đánh giá trên tập validation mỗi epoch train
    'limit_val_set': 0.2, # tỉ lệ giới hạn val dataset
    'overfit_batches': 0, # số batch để ép mô hình overfit (overfit test) 0 - ko dùng
    'earlystopping_min_delta': 0.001, #ngưỡng thay đổi, < delta thì sẽ count + 1, hiện đo theo val_lm_loss
    'earlystopping_patience': 8,  #count = patience thì ngắt dừng train luôn
    'wandb_project': 'GPT_Nemo (multitask, img)', # tên project
    'wandb_key': '6f927fe3835ebcc7bb05946984340ac2c810388e', #key login
    'wandb_run_name': 'Run 2 T Siam: 30k data, siam (huber loss), k60 img features, RL multitask, predict emo text after <gen>, all fill_treatment -100, grad_accum 8' # tên lần chạy (nằm trong project)
}

In [5]:
# !pip install -q --upgrade pytorch-lightning==1.6.3
# !pip install -q --upgrade torchmetrics==0.8.2
# !pip install --upgrade transformers

In [8]:
import torch
from torch import optim, nn
import torchvision

import torchmetrics

import math
from torch.optim.optimizer import Optimizer, required

from torchvision import models, transforms
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torch.nn.functional as F
import pytorch_lightning as pl
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import LearningRateMonitor, Callback
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction 
from nltk.util import ngrams
import json


from transformers import top_k_top_p_filtering
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
# import cv2
from PIL import Image
import os
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
import seaborn as sns
# import plotly.express as px
import datasets
from sklearn.decomposition import PCA
import wandb
import os
from sklearn.metrics.cluster import normalized_mutual_info_score
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import AutoConfig

datasets.disable_progress_bar()
import yaml
tqdm.pandas()

def read_config(path = 'config.yml'):
    with open(path) as f:
        config =  yaml.safe_load(f)
    return config

config = read_config(prefix + 'config.yml')
os.environ["TOKENIZERS_PARALLELISM"] = "false"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [9]:
def init_weights(m):
    """
    Initialize weights for Linear (with bias)
    """
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform(m.weight)
#         torch.nn.init.zeros_(m.bias)
        
class GPT_NEMO(pl.LightningModule):
    """
    Class cho GPT NEMO

    Class này là mô hình chính của mình luôn

    Class này có attributes chứa 
    - object của class mô hình Huggingface Pretrained GPT
    - các lớp Linear cho các task phụ (emotion, intent)

    attrs:
        gpt (huggingface AutoModelForCausalLM): chứa mô hình Huggingface pretrained gpt
        img_intent_fc (torch.nn.Linear): các Lớp linear cho phân lớp ý định sinh ảnh
        emo_fc (torch.nn.Linear): các Lớp linear cho phân lớp cảm xúc
        img_features_fc (torch.nn.Linear): lớp Linear để nhận feature ảnh đầu vào và chuyển qua hidden size (đưa vào chung trong context)
        siam_response_gen_img_fc:  đầu vào hidden size, đầu ra hidden size (để foward last hid state của token <gen>, đại diện cho text)
        siam_img_features_fc: (để forward feature ảnh từ response_img_features, đại diện cho ảnh của response) đầu vào img_features_size , đầu ra hidden size
        
        
    
        4 method dưới là của pytorch lightning hỗ trợ việc train
        training_step: nhận batch data (output của CustomDataset__get_item), bước train sẽ làm gì
        validation_step: nhận batch data (output của CustomDataset__get_item), bước val sẽ làm gì
        validation_epoch_end: hết lượt validation thì làm gì, log ra câu example response 
        configure_optimizers: định nghĩa optimizer
        
    """
    def __init__(self, num_intent_classes=2, num_emotion_classes=52, img_intent_dropout=0.2, emo_dropout=0.2, hidden_size = 768, img_features_size = 768):
        """
        Params 
            num_intent_classes: số class cho ý định sinh ảnh
            num_emotion_classes: số class emotion
            img_intent_dropout: tỉ lệ dropout cho các lớp Linear img_intent_fc
            emo_dropout: tỉ lệ dropout cho các lớp Linear emo_fc
            hidden_size: kích thước hidden state của GPT (là đầu input cho 2 đống Linear kia)
        """ 
        self.num_intent_classes = num_intent_classes
        self.num_emotion_classes = num_emotion_classes

        super(GPT_NEMO, self).__init__()
        self.gpt = AutoModelForCausalLM.from_pretrained(params['pretrained_model'])
        self.hidden_size = hidden_size
        
# #         Linear cho ý định sinh ảnh: 2 hidden layers + gelu activation + dropout
#         self.img_intent_fc = nn.Sequential(
#             nn.Linear(768, 256),
#             nn.Mish(),
#             nn.Linear(256, 128),
#             nn.Mish(),
# #             nn.Dropout(img_intent_dropout),
#             nn.Linear(128, num_intent_classes),
#         )
        

#         self.emo_fc =  nn.Sequential(
#             nn.Linear(768, 256),
#             nn.Mish(),
#             nn.Linear(256, 128),
#             nn.Mish(),
# #             nn.Dropout(img_intent_dropout),
#             nn.Linear(128, num_emotion_classes),
#         )


        self.img_intent_fc = nn.Sequential(
            nn.Linear(hidden_size, num_intent_classes, bias = False),
        )
        
        self.emo_fc =  nn.Sequential(
            nn.Linear(hidden_size, num_emotion_classes, bias = False),
        )

        self.img_features_fc = nn.Sequential(
            nn.Linear(img_features_size, hidden_size, bias = True)
        )

        #siam_response_gen_img_fc đầu vào hidden size, đầu ra hidden size (để foward last hid state của token <gen>, đại diện cho text), 
        self.siam_response_gen_img_fc = nn.Sequential(
            nn.Linear(hidden_size, hidden_size, bias = True)
        )

        #Thêm 1 lớp MLP siam_img_features_fc (để forward feature ảnh từ response_img_features, đại diện cho ảnh)
        # đầu vào feature ảnh shape (768), đầu ra hidden size
        self.siam_img_features_fc = nn.Sequential(
            nn.Linear(img_features_size, hidden_size, bias = True)
        )
        
        
        # Khởi tạo weight cho 2 bộ Linear
        self.img_intent_fc.apply(init_weights)
        self.emo_fc.apply(init_weights)
        
        # image intent metrics
        self.train_intent_accuracy = torchmetrics.Accuracy(num_classes = self.num_intent_classes, average = 'macro')
        self.val_intent_accuracy = torchmetrics.Accuracy(num_classes = self.num_intent_classes, average = 'macro')
        
        self.train_intent_f1 = torchmetrics.F1Score(num_classes = self.num_intent_classes, average = 'macro')
        self.val_intent_f1 = torchmetrics.F1Score(num_classes = self.num_intent_classes, average = 'macro')
        
        # emo metrics
        self.train_emo_accuracy = torchmetrics.Accuracy(num_classes = self.num_emotion_classes, average = 'macro')
        self.val_emo_accuracy = torchmetrics.Accuracy(num_classes = self.num_emotion_classes, average = 'macro')
        
        self.train_emo_f1 = torchmetrics.F1Score(num_classes = self.num_emotion_classes, average = 'macro')
        self.val_emo_f1 = torchmetrics.F1Score(num_classes = self.num_emotion_classes, average = 'macro')
        
        
        

        # Pytorch-Lightning-----------------------------
        # Log your LightningModule hyperparameters
        self.save_hyperparameters()
    


    def forward(self, inputs_embeds, 
                    attention_mask = None, 
                    labels = None, 
                    token_type_ids = None, 
                    gen_indices = None, 
                    response_img_features= None):
        """
        TODO: hàm forward input qua mô hình. 
        Các mô tả params và returns chi tiết hơn ở link
        https://huggingface.co/docs/transformers/model_doc/gpt_neo#transformers.GPTNeoForCausalLM

        Thật sự ở attribute self.gpt đã có method forward rồi,
        method forward của class GPT NEMO sẽ bao trọn, gọi 
        method foward của self.gpt, sau đó sẽ output thêm kết quả của 
        đầu ra Linear intent và emotion

        Note: Các mô hình là class thừa kế của torch.nn.module thì
            Gọi forward chỉ cần model(input) là được, 
            hong cần phải model.forward(input)

        Params:
            input_embeds (torch.Tensor (batch_size, seqlen, hidden_state))
            token_type_ids (torch.Tensor (batch_size, seqlen)): loại token
            labels (shape như token_type_ids): nhãn tương ứng với input_ids
            gen_indices (batch size, 1): vector index của token <gen>
            response_img_features (batch size, 768): feature ảnh của lượt response. Nếu None thì tự fill bằng vector 1 * treatment
        Returns:
            dictionary chứa
                - lm_outputs (dict): outputs của mô hình ngôn ngữ, chứa các items 
                        https://huggingface.co/docs/transformers/model_doc/gpt_neo#transformers.GPTNeoForCausalLM
                    - logits (torch.Tensor): điểm prediction (seqlen, vocab size)
                    - loss (torch.Tensor): loss cho language model (tham số input labels khác None)
                    - hidden_states: tất cả lớp hidden states
            
                - img_intent_logits: điểm prediction (logits) khi forward last hid state token gen qua img_intent_fc
                - emo_logits: điểm prediction (logits) khi forward last hid state token gen qua emo_fc


        """
        lm_outputs = self.gpt( output_hidden_states = True,
                    inputs_embeds = inputs_embeds,
                    labels = labels, 
                    token_type_ids = token_type_ids, 
                    attention_mask = attention_mask)
        
        last_hidden_state = lm_outputs.hidden_states[-1]
        
        # Điều kiện nếu input có gen_indices
        if gen_indices is not None:

            # last hidden state của <gen>
            gen_ids =  gen_indices.reshape(-1, 1)
            gen_ids = gen_ids.repeat(1, self.hidden_size).view(last_hidden_state.shape[0], 1, self.hidden_size)
        
            gen_last_hstate = torch.gather(last_hidden_state, 1, gen_ids).reshape(-1, self.hidden_size)
            # shape (batch size, 1, hidden size)

            # điểm prediction (logits) khi forward last hid state token gen qua img_intent_fc
            img_intent_logits = self.img_intent_fc(gen_last_hstate)

            # logits khi forward last hid state token gen qua emo_fc
            emo_logits = self.emo_fc(gen_last_hstate) 

            # siamese fake: siam_response_gen_img_fc_out, siam_img_features_fc_out
            #siam_response_gen_img_fc đầu vào hidden size, đầu ra hidden size (để foward last hid state của token <gen>, đại diện cho text), 
            siam_response_gen_img_fc_out = self.siam_response_gen_img_fc(gen_last_hstate)
            

            nan_treatment_num = -100
            if response_img_features is None:
                response_img_features = torch.ones((params['batch_size'], params['img_features_size'])) * nan_treatment_num
                response_img_features = response_img_features.to(device)
                
            #siam_img_features_fc đầu vào img_features_size, đầu ra hidden size
            siam_img_features_fc_out = self.siam_img_features_fc(response_img_features)


        # !!!!!!! Lỗi: Shape của img_intent_logits, emo_logits nên reshape thành 2 chiều, chiếu thứ 2 bị thừa
        # chiều 1: batch size, 
        # chiều 2: 1, thừa ròi
        # chiều 3: số lượng class

            return {'lm_outputs': lm_outputs, 
                    'img_intent_logits': img_intent_logits.reshape(-1, self.num_intent_classes), 
                    'emo_logits': emo_logits.reshape(-1, self.num_emotion_classes),
                    'siam_response_gen_img_fc_out': siam_response_gen_img_fc_out.reshape(-1, self.hidden_size),
                    'siam_img_features_fc_out': siam_img_features_fc_out.reshape(-1, self.hidden_size)
                }
        
        else:
            return {'lm_outputs': lm_outputs}
    
    def training_step(self, batch, batch_idx):
        inputs_embeds = batch['inputs_embeds']
        token_type_ids = batch['token_type_ids']
        labels = batch['labels']
        response_img_features = batch['response_img_features']

        gen_indices = batch['gen_index']
        outputs = self.forward(inputs_embeds = inputs_embeds, 
                    token_type_ids = token_type_ids, 
                    labels = labels, 
                    gen_indices = gen_indices,
                    response_img_features= response_img_features)

        lm_loss = outputs['lm_outputs']['loss']
        # print(outputs['img_intent_logits'].shape)
        # print(batch['response_has_img_labels'].shape)
        # print(batch['response_has_img_labels'])

        # Lỗi do phần reshape trong method forward của Model
        # Img intent clf
        img_intent_loss = img_intent_CE(outputs['img_intent_logits'], batch['response_has_img_labels'])
        # Emo clf
        emo_loss = emo_CE(outputs['emo_logits'], batch['response_emotion_id_labels'])

        # Siamese fake
        siam_loss = huber_loss(outputs['siam_response_gen_img_fc_out'], outputs['siam_img_features_fc_out'])


        # Multitask loss
        num_tasks = 4

        # code dòng này từ paper A CLOSER LOOK AT LOSS WEIGHTING IN MULTITASK LEARNING
        random_weights = F.softmax(torch.randn(num_tasks), dim=-1)
        
#         total_loss = torch.sum(multitask_loss * random_weights)
        total_loss = random_weights[0] * lm_loss + random_weights[1] * img_intent_loss + random_weights[2] * emo_loss + random_weights[3] * siam_loss

        
        # log lên wandb
        # Loss
        self.log("train/lm_loss", lm_loss, on_step=True, on_epoch = True)
        self.log("train/img_intent_loss", img_intent_loss, on_step=True, on_epoch = True)
        self.log("train/emo_loss", emo_loss, on_step=True, on_epoch = True)
        self.log("train/siam_loss", siam_loss, on_step = True, on_epoch = True)
        self.log("train/multitask_loss", total_loss, on_step=True, on_epoch = True)
        
    
        # Accuracy, f1
        img_intent_logits = outputs['img_intent_logits']
        img_intent_labels = batch['response_has_img_labels']
        emo_logits = outputs['emo_logits']
        emo_labels = batch['response_emotion_id_labels']
        
        self.train_intent_accuracy(img_intent_logits, img_intent_labels)
        self.log("train/accuracy_img_intent", self.train_intent_accuracy, on_step = True, on_epoch = True)
        
        self.train_intent_f1(img_intent_logits, img_intent_labels)
        self.log("train/f1_img_intent", self.train_intent_f1, on_step = True, on_epoch = True)
        
        # emo metrics
        self.train_emo_accuracy(emo_logits, emo_labels)
        self.log("train/accuracy_emo", self.train_emo_accuracy, on_step = True, on_epoch = True)

        self.train_emo_f1(emo_logits, emo_labels)
        self.log("train/f1_emo", self.train_emo_f1, on_step = True, on_epoch = True)

        
        
        
#         print(f"Epoch {self.current_epoch}-{batch_idx} - img_intent_logits: {torch.argmax(outputs['img_intent_logits'], dim=1)}")
        return total_loss
#         return img_intent_loss

    def validation_step(self, batch, batch_idx):
        inputs_embeds = batch['inputs_embeds']
        token_type_ids = batch['token_type_ids']
        labels = batch['labels']
        response_img_features = batch['response_img_features']

        gen_indices = batch['gen_index']
        outputs = self.forward(inputs_embeds = inputs_embeds, 
                    token_type_ids = token_type_ids, 
                    labels = labels, 
                    gen_indices = gen_indices,
                    response_img_features= response_img_features)

        lm_loss = outputs['lm_outputs']['loss']
        # print(outputs['img_intent_logits'].shape)
        # print(batch['response_has_img_labels'].shape)
        # print(batch['response_has_img_labels'])

        # Lỗi do phần reshape trong method forward của Model
        img_intent_CE = nn.CrossEntropyLoss()
        emo_CE = nn.CrossEntropyLoss()
        # Img intent clf
        img_intent_loss = img_intent_CE(outputs['img_intent_logits'], batch['response_has_img_labels'])
        # Emo clf
        emo_loss = emo_CE(outputs['emo_logits'], batch['response_emotion_id_labels'])

        # siam loss
        siam_loss = huber_loss(outputs['siam_response_gen_img_fc_out'], outputs['siam_img_features_fc_out'])
        
        # Multitask loss
        num_tasks = 4
#         multitask_loss = torch.tensor([lm_loss, img_intent_loss, emo_loss]).reshape(1, num_tasks)
        # code dòng này từ paper A CLOSER LOOK AT LOSS WEIGHTING IN MULTITASK LEARNING
#         random_weights = F.softmax(torch.randn(num_tasks), dim=-1)
        
        # total_loss = torch.sum(multitask_loss * random_weights)
        total_loss = lm_loss + img_intent_loss + emo_loss + siam_loss
        
        # log lên wandb
        # Loss
        self.log("val/lm_loss", lm_loss, on_step=True, on_epoch = True)
        self.log("val/img_intent_loss", img_intent_loss, on_step=True, on_epoch = True)
        self.log("val/emo_loss", emo_loss, on_step=True, on_epoch = True)
        self.log("val/siam_loss", siam_loss, on_step = True, on_epoch = True)
        self.log("val/multitask_loss", total_loss, on_step=True, on_epoch = True)
        
        # Accuracy, f1
        img_intent_logits = outputs['img_intent_logits']
        img_intent_labels = batch['response_has_img_labels']
        emo_logits = outputs['emo_logits']
        emo_labels = batch['response_emotion_id_labels']
        
        self.val_intent_accuracy(img_intent_logits, img_intent_labels)
        self.log("val/accuracy_img_intent", self.val_intent_accuracy, on_step = True, on_epoch = True)
        
        self.val_intent_f1(img_intent_logits, img_intent_labels)
        self.log("val/f1_img_intent", self.val_intent_f1, on_step = True, on_epoch = True)
        
        # emo metrics
        self.val_emo_accuracy(emo_logits, emo_labels)
        self.log("val/accuracy_emo", self.val_emo_accuracy, on_step = True, on_epoch = True)

        self.val_emo_f1(emo_logits, emo_labels)
        self.log("val/f1_emo", self.val_emo_f1, on_step = True, on_epoch = True)
        
        

        
    def validation_epoch_end(self, outputs):
        # log 1 câu example response, thử context của mình
        columns = ["epoch", "context", "response", "img_intent", "emo"]
        example_context = "<speaker1>Lost his head<|endoftext|>happy<speaker2>" 
        example_output = generate(example_context, max_length= 20, top_k= 200, top_p= 0.8, temperature = 0.95)
        example_response = example_output['response']
        
        img_intent = example_output['img_intent']
        emo = example_output['emo']
        
        data = [[self.current_epoch, example_context, example_response, img_intent, emo]]
        
        # log lên table wandb
        wandb_logger.log_text(key="example response", columns=columns, data=data)
        print(example_response, img_intent, emo)
        
                
    def configure_optimizers(self):
        optimizer = optim.AdamW(self.parameters(), lr = params['lr'], weight_decay= params['weight_decay'])
        
        
        scheduler = optim.lr_scheduler.OneCycleLR(optimizer, 
                                                  max_lr= params['max_lr'], 
                                                  total_steps=self.trainer.estimated_stepping_batches)
        
        
#         optimizer = Ranger(self.parameters(), lr = params['lr'])
#         T_max: bn step thì tăng dần lên. 
#         scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer,
#                               T_max= self.trainer.estimated_stepping_batches // params['epochs'], 
#                               eta_min=1e-4,
#                               last_epoch=-1)
#         scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[5, 6], gamma=0.1)

        
        scheduler_config = {
            # REQUIRED: The scheduler instance
            "scheduler": scheduler,
            # The unit of the scheduler's step size, could also be 'step'.
            # 'epoch' updates the scheduler on epoch end whereas 'step'
            # updates it after a optimizer update.
            "interval": "step",
            # How many epochs/steps should pass between calls to
            # `scheduler.step()`. 1 corresponds to updating the learning
            # rate after every epoch/step.
            "frequency": 1,
        }
        
        
        
        return {'optimizer': optimizer, 
                'lr_scheduler': scheduler_config
               }

def add_special_tokens(tokenizer, model):
    SPECIAL_TOKENS_DICT = {
        'pad_token': '<pad>',
        'additional_special_tokens': ['<speaker1>', '<speaker2>', '<gen>', '<emo>', '<img>']
    }

    tokenizer.add_special_tokens(SPECIAL_TOKENS_DICT)
    # add token mới xong thì tăng số cột (số token) của ma trận token embeddings lên 
    model.gpt.resize_token_embeddings(len(tokenizer))
    
def preprocess_df_format(df):
    """ Tạo df với format. Hàm này nằm trong method init() của class CustomDataset
    Return được pd.dataframe (mỗi lượt nói là 1 datapoint, )
    - context: ngữ cảnh (dùng cumsum để cộng dồn txt theo dialogue_id) (1 -> t-1)
    - response: lượt trả lời hiện tại (t)
    - speaker_id: lượt hiện tại là của ai (speaker1, speaker2)
    - response_has_img: response có ảnh hay ko
    - response_emotion_id: emotion của response tương ứng
    - img_id: ảnh cuối của lượt context (dùng để cộng dồn history_imgs sau này)
    - response_img_id: ảnh của lượt response (dùng để model học task simaese fake)
    """

    # reset index lấy dialogue_id
    tiny_df = df.reset_index().copy()
    tiny_df.emotion_id = tiny_df.emotion_id.fillna('normal')
    
    # speaker_id: Đổi [speaker1] --> <speaker1> và tương tự cho speaker2
    # => cho đúng định dạng special token
    tiny_df['speaker_id'] = tiny_df['speaker_id'].map({'[speaker1]': '<speaker1>', '[speaker2]': '<speaker2>'})

    # context: groupby theo dialogue_id, cộng dồn txt mỗi lượt nói tạo context
    temp = tiny_df.groupby('dialogue_id')\
                .apply(lambda x: (x['speaker_id'] + x['txt'] + '<|endoftext|>' + x['emotion_id'] + '<img>').cumsum().str.strip())

    # Tạo dataframe mới
    # Cột context: cộng dồn cột txt tạo thành context, xem là quá khứ (0 -> t-1)
    # Cột response: sẽ là đại diện cho câu trả hiện tại (t)
    # Cột response = Cột txt cũ nhưng tịnh tiến lên (shift up) 1 giá trị, câu response là câu txt tương lai (dòng kế tiếp)
    response = list(tiny_df['txt'].iloc[1:]) # vị tri 0 = vị trí 1, vị trí 1 = vị trí 2, vị trí n = vị trí n + 1 (lỗi này ở dưới sẽ nói)
    response.append(np.nan) # vị trí cuối của dataset sẽ là nan

    # tương tự với cột speaker_id luôn, speaker_id đại diện cho response
    speaker_id = list(tiny_df['speaker_id'].iloc[1:])
    speaker_id.append(np.nan) # vị trí cuối cùng của dataset sẽ là nan 
                            # nên ở dưới sẽ dropna các dòng chứa 
                                # np.nan ở cột response, speaker_id

    # has_img
    has_img = list(tiny_df['img_id'].iloc[1:].notnull().astype(int))
    has_img.append(np.nan)

    # img_id: cột này để cộng dồn history_img
    # img_id
    img_id = tiny_df['img_id']
    # img_id = list(tiny_df['img_id'].iloc[1:])
    # img_id.append(np.nan)

    # response_img_id: cột này để cho task siamese fake, dự đoán ảnh đi kèm của response 
    response_img_id = list(tiny_df['img_id'].iloc[1:])
    response_img_id.append(np.nan)

    # emotion_id
    emo2id_dict = pd.read_json(prefix + "data/emo2id.json", orient = 'index')
    emo2id_dict = emo2id_dict.to_dict()[0]
    
    emotion = list(tiny_df['emotion_id'].iloc[1:].map(emo2id_dict))
    emotion.append(np.nan)

    # Cột dialogue_id, img_id, emotion_id giữ nguyên, đưa vào sau này có thể xài
    temp = pd.DataFrame({'dialogue_id': tiny_df.dialogue_id, 
                        'speaker_id': speaker_id,
                        'context': temp.values, 
                        'response': response, 
                        'response_has_img': has_img, 
                        'response_emotion_id': emotion, 
                        'img_id': img_id,
                        'response_emo_text': tiny_df.emotion_id,
                        'response_img_id': response_img_id})
    temp = temp.dropna(subset = ['response', 'speaker_id'])

    # !!!! sửa lỗi response có thể bị mất <speaker_id> nếu quá dài và bị cắt từ trái qua mất
    # Thêm token <speaker_id> cho lượt response ở đây luôn
    temp['context'] = temp['context'] + temp['speaker_id']

    # Thêm token <|endoftext|> ở cuối response, và cả <gen>, và cả emo text
    temp['response'] = temp['response'] + '<|endoftext|>' +  '<gen>' + temp['response_emo_text']

    # Đổi cột response_img_id thành str (nan thành 'nan')
    temp['response_img_id'] = temp['response_img_id'].apply(lambda x: str(x))

    # Lấy những dòng cuối của mỗi dialogue, mất k dòng với k là số lượt dialogue
    cursed_index = temp.groupby('dialogue_id').tail(1).index
    temp = temp[~temp.index.isin(cursed_index)]
    return temp

def create_history_imgs(df, print_info=False):
    # dùng np.nan khum được ta, lỗi gì đó khó hiểu
    def isNaN(num):
        return num!= num

    temp = df.copy()
    temp['img_id'] = temp['img_id'].apply(lambda x: str(x) if not isNaN(x) else 'nan')

    # 1 dòng thoi quá chiến luôn =)))
    history_imgs = temp.groupby('dialogue_id')\
                    .apply(lambda x: (x['img_id'] + ' ').cumsum().str.strip().str.split(' ')).values

    if print_info:
        print("history: ", history_imgs[:10])

    temp['history_imgs'] = history_imgs
    return temp

def build_token_type_ids(context):
    """ dùng trong hàm prepare_inputs() và generate()
    return ra list token_type_ids từ input_ids (context)
    
    # token_type_ids: khó khăn nhất. Dùng code hư hỏng tí thoi chứ sao giờ =))
    # Loop qua từng token trong input_ids
        # Nếu token là <speaker1> thì push <speaker1> dô token_type_ids
        # Nếu token là <speaker2> thì push <speaker2> dô token_type_ids
        # Nếu token là <pad> thì push <pad> dô token_type_ids
        # Nếu token là bình thường thì sẽ push <speaker1> hoặc 2 tuỳ theo loại token lượt trước đó
        # Nếu token là <gen> thì push <gen> dô token_type_ids
    """
    token_type_ids = []
    # 1: token <speaker1>, 0: token <speaker2>, -1: token <pad>, -2: token <gen>, -3: token <img>
    is_speaker1 = 1 
    
    list_context = list(context)
    for i, token in enumerate(list_context):
        if token == user1_token:
            is_speaker1 = 1
        elif token == user2_token:
            is_speaker1 = 0
        elif token == pad_token:
            is_speaker1 = -1
        elif token == gen_token:
            is_speaker1 = -2
        elif token == img_token:
            is_speaker1 = -3
        
        if is_speaker1 == 1:
            token_type = user1_token
        elif is_speaker1 == 0:
            token_type = user2_token
        elif is_speaker1 == -1:
            token_type = pad_token
        elif is_speaker1 == -2:
            token_type = gen_token
        elif is_speaker1 == -3:
            token_type = img_token
        
        #bỏ đk sau nếu response ko cần thêm emotion
        if list_context[i-1] == eos_token and list_context[i] != gen_token: 
            token_type = emo_token
            # print(token)
        # Nếu ở trước là gen-1 thì tiếp theo sẽ là emo text
        if list_context[i-1] == gen_token:
            token_type = emo_token
            
        token_type_ids.append(token_type)
    
    return token_type_ids



def prepare_inputs(row, print_info = False, seqlen = 512, 
                    context_len = 480, response_len = 32):
    """
    Chuẩn bị input_ids, token_type_ids, labels từ 1 dòng data
    Format 1 dòng data phải có định dạng như output của hàm preprocess_df_format()
    - input_ids: index cột của Word embedding vocab
    - token_type_ids: loại token tương ứng với input_ids, vị trí token <gen> là <gen>
    - labels: nhãn để tính loss (chỉ tính trên phần response, 
                các vị trí còn lại gắn -100 là huggingface tự xử lí)

    Params:
        row (pd.Series): 1 dòng trong dataframe
        seqlen (int): độ dài của chuỗi input (mặc định 512)
        context_len (int): độ dài chuỗi tối đa cho context (mặc định 480)
        response_len (int): độ dài chuỗi tối đa response (mặc định 32)
            phải đảm bảo: seqlen = context_len + response
        print_info (bool): in mấy tensor ra 
            (set True nếu chạy 1 lần để in quá trình tính, bỏ vô 
            hàm apply chạy nhiều lần toang notebook á =))    

    Returns:
        dictionary
            input_ids (torch.Tensor (1, seqlen))
            token_type_ids (torch.Tensor (1, seqlen))
            labels (torch.Tensor (1, seqlen))
            gen_index (int): vị trí của token <gen>
            response_has_img: lượt response có ảnh hay k
            response_emotion_id: emotion cho lượt response
            history_imgs: lịch sử ảnh cho dòng hiện tại
            response_img_id: ảnh cho lượt response (để model học task siamese fake)
    """

    # tokenize: k padding chỉ truncate nếu vượt độ dài (truncate từ trái sang). 
    # độ dài mặc định: context (480) + response (32) = seqlen (512). 
    # Theo thống kê hôm bữa thì 1 lượt trả lời độ dài cỡ chừng 32 (64), có thể tăng lên đây mới thử thoi
    context = tokenizer.encode(row['context'], max_length = context_len, truncation = True) 
    response = tokenizer.encode(row['response'], max_length = response_len, truncation = True)

    # đảm bảo context mở đầu bằng token <speaker1> hoặc <speaker2> 
    # (với context dài đã bị cắt từ trái qua, thì nó sẽ mở đầu ở bất kì token nào)
    # Như vậy sẽ ảnh hưởng đến việc học 
    if context[0] != user1_token and context[0] != user2_token:
        start_index = 0
        for i, token in enumerate(context):
            if token == user1_token or token == user2_token:
                start_index = i 
                break
        context = context[start_index:]

    # input_ids----------
    # Phép cộng list: [23] + [4] = [23, 4]
    # Phép nhân list: [234] * 2 = [234, 234]
    # Context + response, sau đó nhân nối token <pad> cho tới 
            # phần độ dài còn lại (seqlen - len(context) - len(response) 
            # đạt đủ seqlen

    input_ids = context + response + [pad_token] * (seqlen - len(context) - len(response))
    
    assert len(input_ids) == seqlen

    # gen_index: tìm index của list input_ids có giá trị (int)
    # gen_token: index của token gen trong vocab
    gen_index = input_ids.index(gen_token)

    # labels------------- 
        # gắn -100 (theo như docs thì token nào -100 sẽ hong tính loss):
        # Hong tính loss toàn bộ phần context, 
        # response thì hong tính loss cho token <gen> trở về sau
        
        # => response[:-1] + [-100] cho token <gen>. Update:: cho phép token <gen> sinh text emo
    labels = [-100] * len(context) + response + [-100] * (seqlen - len(context) - len(response))
    assert len(labels) == seqlen

    # token_type_ids: khó khăn nhất. Dùng code hư hỏng tí thoi chứ sao giờ =))
    # Loop qua từng token trong input_ids
        # Nếu token là <speaker1> thì push <speaker1> dô token_type_ids
        # Nếu token là <speaker2> thì push <speaker2> dô token_type_ids
        # Nếu token là <pad> thì push <pad> dô token_type_ids
        # Nếu token là bình thường thì sẽ push <speaker1> hoặc 2 tuỳ theo loại token lượt trước đó
        # Nếu token là <gen> thì push <gen> dô token_type_ids
    token_type_ids = build_token_type_ids(input_ids)
    
    if print_info:
        print("context: ", context)
        print("response: ", response)
        print("context + response: ", input_ids)
        # print("len of context + response: ", len(input_ids))
        print("labels: ", labels)
        print("token type ids: ", token_type_ids)
        # print("len of labels: ", len(labels))
        print("response_has_img: ", row['response_has_img'])
        print("response_emotion_id: ", row['response_emotion_id'])
        print("history_imgs: ", row['history_imgs'])
        print("response_img_id: ", row['response_img_id'])


    # Kiểm tra với dataframe, token 50258 là <speaker1>, token 50259 là <speaker2>, 50257 là <pad>
    # display(pd.DataFrame({'type': token_type_ids, 'input': input_ids}))
    # Đã kiểm tra song song 2 cột bằng mắt thì chuẩn hết luôn ròi

     
    return {'input_ids': input_ids, 
            'token_type_ids': token_type_ids, 
            'labels': labels, 
            'response_has_img': row['response_has_img'], 
            'response_emotion_id': row['response_emotion_id'], 
            'gen_index': gen_index, 
            'history_imgs': row['history_imgs'],
            'response_img_id': row['response_img_id']}


class NemoDataset(Dataset):
    """
    Class CustomDataset cho mô hình
    Attrs:
        df: dataframe đã được chuẩn bị đúng format (output hàm preprocess_df_format)
        inputs: input chuẩn đã có input_ids, token_type_ids, labels (output của apply hàm prepare_inputs)
    Methods:
        __init__(self, data = path_to_pickle)
        __getitem__(self, idx)
    """

    def __init__(self, data= config['path']['train_preprocessed_path']):
        """
        Attrs:
            df: dataframe đã được chuẩn bị đúng format
            inputs: input chuẩn đã có input_ids, token_type_ids, labels
        Params:
            data (string): Path to the pickle file.
        """
        # Đang thử nghiệm trên lượng nhỏ data nên chỉ lấy num_samples dòng dữ liệu
        self.df = pd.read_pickle(data).iloc[:params['num_samples']]

        # Replace nan emotion
        self.df.emotion_id = self.df.emotion_id.fillna('normal')
        
        # Chuẩn bị df chuẩn format
        print("preprocessing...")
        self.df = preprocess_df_format(self.df)

        # Lỗi phát hiện trong quá trình train, chạy training loop mới thấy
        # Do mình có delete dòng (dòng cuối mỗi dialogue_id) ở hàm preprocess_df_format 
        # nên index nó bị lệch --> __getitem(idx) bị lỗi =))
        self.df = self.df.reset_index(drop = True)

        # Bổ sung history imgs
        self.df = create_history_imgs(self.df)

        # Chuẩn bị các các dạng inputs cần thiết
        print("preparing inputs...")
        self.inputs = self.df.progress_apply(lambda x: prepare_inputs(x, 
                                                                seqlen= params['seqlen'], 
                                                                context_len= params['context_len'], 
                                                                response_len= params['response_len'])
                                                                , axis = 1)

        # load dictionary json img_features
        with open(prefix + params['img_features_json']) as json_file:
            self.img_id2features = json.load(json_file)


    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        """
        Lấy data từ (pd.series) 1 dòng dữ liệu cụ thể theo index, 
        để sau này forward nó qua mô hình

        data 1 dòng: input_ids, token_type_ids, labels, gen_index

        inputs_embeds (torch.tensor (batch_size là 1, seqlen, hidden states)): 
            Dùng model.gpt.get_input_embeddings() để lấy word embedding của input_ids
        
        gen_last_hstate (torch.tensor(batch_size là 1, hidden states))
            Dùng inputs_embeds, lấy last hidden state của <gen> dựa vào gen_index (truy xuất chiều t2, t3)

        Params:
            idx (int): code mẫu, chỉ mục của item cần lấy
            self.inputs: dictionary chứa data 1 dòng (mỗi item là 1 list)

        Returns:
            dictionary chứa:
                inputs_embeds (torch.tensor)
                token_type_ids (torch.tensor)
                labels (torch.tensor): nhãn cho lm
                response_has_img_labels (torch.tensor): nhãn cho ý định sinh ảnh
                response_emotion_id (torch.tensor): nhãn cho phan lớp cảm xúc
                gen_index (torch.tensor):  index của token gen của dòng data idx 
                response_img_features (torch.tensor): feature ảnh của response (shape feature từ json)

        """
        items = self.inputs[idx] # lấy được dòng

        # input_ids, token_type_ids, labels đã được mình chuẩn bị sẵn cho từng dòng (ở method init)
        input_ids = torch.Tensor(items['input_ids']).long().to(device)
        token_type_ids = torch.Tensor(items['token_type_ids']).long()
        labels = torch.Tensor(items['labels']).long()

        # lấy gen_index 
        # mục đích: khi forward mô hình sẽ được last hidden state của tất cả token
        # Lấy ra last hidden state của gen thoi
        gen_index = torch.tensor(items['gen_index']).long()

        response_has_img_labels = torch.tensor(items['response_has_img']).long()
        response_emotion_id_labels = torch.tensor(items['response_emotion_id']).long()

        # Lưu ý bước này phải bật model.eval(), để bỏ qua dropout và batchnorm
        # bỏ vô torch.no_grad để ko tính gradient cho hàm này.
        # Nếu hong có, mình sẽ kiểu forward mô hình 2 lần là toang bước tính gradient luôn

#         model.eval()
#         with torch.no_grad():
        inputs_embeds = model.gpt.get_input_embeddings()(input_ids)

        # bổ sung history imgs-----------------------------------------
        history_imgs = items['history_imgs'] #lấy ra từ row

        # Kích thước số feature ảnh
        hidden_size = params['hidden_size']
        img_features_size = params['img_features_size']

        # Khởi tạo tensor rỗng (len(history_imgs), hidden size): mỗi dòng là 1 token emb, mỗi cột là hidden feature gpt
        img_features = torch.zeros((len(history_imgs), img_features_size))

        for idx, img in enumerate(history_imgs):
            # Nếu ảnh khác 'nan', lấy feature ảnh từ dict, chuyển thành tensor có shape (hidden size)
            if img != 'nan':
                img_feature = torch.tensor(self.img_id2features[img])
            # Nếu ảnh là 'nan', tensor zeros shape (hidden size)
            else:
                nan_treatment_num = -100
                img_feature = torch.ones(img_features_size) * nan_treatment_num
            
            # Reshape img_feature thành shape (1, hidden size), để concat với img_features (cần 2 chiều)
            img_feature = img_feature.reshape(1, img_features_size)

            # Thay giá trị dòng thứ idx của img_features
            img_features[idx, :] = img_feature

        # Thay thế vector (feature ảnh) dô dòng token embedding của các token <img> 
        # Đi ngược từ phải về trái, ngược lối như đã hứa =((
        # Do input_ids và inputs_embeds nó tương ứng với nhau từng token 1, nên duyệt theo
            # input_ids sẽ tiện hơn, và replace thì ở inputs_embeds
        # Trước khi replace sẽ đưa ảnh qua lớp Linear img_features_fc (img_features_size, hidden size): 
            #để mô hình học cách chuyển feature ảnh qua kích thước hidden size

        # đẩy qua device gpu (cùng device với inputs_embeds, inputs_embeds đang ở gpu)
        img_features = img_features.to(device)
        
        # biến count để chạy cho img_features, chạy lùi luôn mới dị =))
        count = -1
        for idx, token in reversed(list(enumerate(input_ids))):
            if token == img_token:
                # print(tokenizer.decode(token))
                # print("inputs_embed[idx]: ", inputs_embeds[idx, :][:5]) #in 5 ptu tại dài quá
                # print("img feature[idx]: ", img_features[count, :][:5])

                proj_img_features = model.img_features_fc(img_features[count, :])
                inputs_embeds[idx, :]= proj_img_features
                count -= 1
        
        #---------------------------------
        # response_img_id: lấy feature ảnh của lượt response (cho model học task siamese fake)
        response_img_id = items['response_img_id']
        nan_treatment_num = -100

        if response_img_id == 'nan':
            response_img_features = torch.ones((img_features_size)) * nan_treatment_num
        else:
            response_img_features = torch.tensor(self.img_id2features[response_img_id])
            
        
        return {'inputs_embeds': inputs_embeds, 
                'token_type_ids': token_type_ids, 
                'labels': labels,
                'response_has_img_labels': response_has_img_labels,
                'response_emotion_id_labels': response_emotion_id_labels,
                'gen_index': gen_index,
                'response_img_features': response_img_features
                }

# Thuận toán sinh câu, code tay mệt chết đi sống lại như đang đẻ =))  

def decode_top_k_top_p(logits, max_length, top_k=0, top_p=1):
    """Decoding algorithm top p, top k, 
    from https://gist.github.com/thomwolf/1a5a29f6962089e871b94cbd09daf317

        Params:
            logits: lấy từ output của mô hình - outputs.logits (vocab size)
            max_length
            top_k >0: keep only top k tokens with highest probability (top-k filtering).
            top_p >0.0: keep the top tokens with cumulative probability >= top_p (nucleus filtering).
    """
    filtered_logits = top_k_top_p_filtering(logits, top_k= top_k, top_p= top_p)
    probabilities = F.softmax(filtered_logits, dim= -1)
    next_token = torch.multinomial(probabilities, 1)
    return next_token


def generate(txt, context_length = params['seqlen'], max_length=20, top_k=0, top_p=1, temperature = 1):
    """
    hàm sinh câu
    Params:
            txt (str): input sentence (context). 
                String này đã có emotion, ko có <gen> ở cuối luôn (<gen> ở phần response mà response chưa sinh)

            context_length: độ dài tối đa cho context (mặc định là param seqlen)
            max_length (int): max length
            top_k >0: keep only top k tokens with highest probability (top-k filtering).
            top_p >0.0: keep the top tokens with cumulative probability >= top_p (nucleus filtering).
            temperature (float [0,1] ): càng thấp thì càng sinh mấy từ phổ biến
    
    Returns:
            dictionary
                response: Câu được sinh ra (str) (không kèm theo context). Có <gen> ở cuối sẵn
                img_intent: ý định sinh ảnh (int)
                emo: lớp cảm xúc (int)
    """
    # Bật chế độ eval
    model.eval()

    # Đảm bảo context bắt đầu bằng <speaker1> hoặc <speaker2>
    context = tokenizer(txt, truncation = True, max_length = context_length)['input_ids']
    if context[0] != user1_token and context[0] != user2_token:
        start_index = 0
        for i, token in enumerate(context):
            if token == user1_token or token == user2_token:
                start_index = i 
                break
        context = context[start_index:]
    
    # độ dài giới hạn để dừng sinh câu
    limit_len = len(context) + max_length
    
    # list các input_ids token
    sentence_ids = context
    response_start_index = len(sentence_ids)
    
    while True:# mỗi vòng lặp
        # tính token_type_ids cho 
        token_type_ids = build_token_type_ids(sentence_ids)
            
        # cho input forward qua model
        with torch.no_grad(): 
            token_type_ids = torch.tensor(token_type_ids).to(device)
            input_ids = torch.tensor(sentence_ids).to(device).reshape(1, -1)
            inputs_embeds = model.gpt.get_input_embeddings()(input_ids)
            outputs = model(inputs_embeds = inputs_embeds, token_type_ids=token_type_ids)
    
        # prediction score
        logits = outputs['lm_outputs'].logits[0] #[0] do chiều đầu tiên là batch size, phải bỏ

        # chia cho temperature
        logits = logits / temperature

        # lấy next token theo top_k top_p
        next_token = decode_top_k_top_p(logits, max_length=max_length, top_k = top_k, top_p = top_p)
        
        # lấy token cuối (mới đẻ), chuyển từ tensor thành list
        next_token = next_token[-1].tolist()[0] 

        # thêm token mới đẻ dô list các input_ids
        sentence_ids.append(next_token)

        # đk dừng loop, eos, pad, chạm max_length
        if  (next_token == tokenizer.eos_token_id) or (next_token == tokenizer.pad_token_id) or len(sentence_ids) >= limit_len:
            break

    # Thêm token <gen> dô cuối câu đã sinh
    sentence_ids.append(gen_token)
    # Tạo lại token_type_ids mới
    token_type_ids = build_token_type_ids(sentence_ids)
    
    response = sentence_ids.copy()

    # đổi input_ids qua tensor
    sentence_ids = torch.tensor(sentence_ids)
    # đổi token type_ids qua tensor
    token_type_ids = torch.tensor(token_type_ids)

    # đổi gen_index qua tensor
    gen_index = torch.tensor(sentence_ids.shape[0] - 1)

    # Reshape 3 bé cho có batch size 1
    sentence_ids = sentence_ids.reshape(1, -1).to(device)
    token_type_ids = token_type_ids.reshape(1, -1).to(device)
    gen_index = gen_index.reshape(1, -1).to(device)

    with torch.no_grad():
        inputs_embeds = model.gpt.get_input_embeddings()(sentence_ids)
        outputs = model(inputs_embeds = inputs_embeds, token_type_ids= token_type_ids, gen_indices = gen_index)
        img_intent_logits = outputs['img_intent_logits']
        emo_logits = outputs['emo_logits']

    # !!!!!! return ra class cụ thể luôn
    # print(img_intent_logits)
    # print(emo_logits)

    img_intent_pred = torch.argmax(img_intent_logits, dim=1)
    emo_pred = torch.argmax(emo_logits, dim = 1)

    return {
            'response': tokenizer.decode(response[response_start_index:]).strip(),
            'img_intent': int(img_intent_pred),
            'emo': int(emo_pred)
           }

# !!!!!!! Chưa làm tới phần này    
def create_evaluation_inputs(dataset, limit_val_set):
    """
    Chuẩn bị inputs (predicted response), gold_response (actual response)
    Dùng 2 biến này, tiến hành đánh giá (bleu, distinct) ở Callback tự định nghĩa, sau khi train xong mô hình sẽ chạy
    Params
        dataset (Pytorch CustomDataset): tập để đánh giá (val set or test set) 
        limit_val_set (int or float)
            int: chỉ lấy "limit_val_set" dòng data của tập dataset
            float: chỉ lấy "limit_val_set" lượng data của tập dataset
                set limit_val_set = 1 --> lấy toàn bộ data của dataset
            do chạy hàm generate trên toàn val set khá lâu (3h cho 9k dòng)
    Returns
        list_of_inputs (list of str): input (context) cho quá trình đánh giá, đã được gắn token speaker_id ở cuối
        list_of_gold_responses (list of str): câu response gốc
    """
    if limit_val_set <= 1:
        num_rows = int(len(dataset) * limit_val_set)
    else:
        num_rows = limit_val_set
    temp_df = dataset.df.sample(n= num_rows, random_state = params['random_seed'])
    
    # context + token speaker_id: input cho quá trình generate
    list_of_inputs = list(temp_df['context'] + ' ' + temp_df['speaker_id'])
    # gold_response = response nhưng bỏ token đầu tiên (token speaker_id)
    list_of_gold_responses = list(temp_df['response'].apply(lambda x: x.split(' ', 1)[1]))
    
    return list_of_inputs, list_of_gold_responses

def compute_bleu_n(list_of_preds, list_of_gold_responses, n = 4):
    """
    Tính bleu_n
    """
    def bleu_n(reference, hypothesis, n):
        if n == 2:
            weights = (1./2, 1./2)
        elif n == 3:
            weights = (1./3, 1./3, 1./3)
        elif n == 4:
            weights = (1./4, 1./4, 1./4, 1./4)
        else:
            raise ValueError("bleu_n = 2 or 3 or 4.")

        return corpus_bleu([[reference]], [hypothesis], weights=weights, smoothing_function=SmoothingFunction().method1) 

    res = 0
    for i in range(len(list_of_preds)): 
        x = tokenizer.encode(list_of_preds[i]) 
        y = tokenizer.encode(list_of_gold_responses[i]) 
        res += bleu_n(x, y, n=n) 
    
    return res / len(list_of_preds)
    
def compute_dist_n(list_of_sentences, n):
    """
        Tính dist_n
    """
    def distinct_n_sentence_level(sentence, n):
        """
        Compute distinct-N for a single sentence.
        :param sentence: a list of words.
        :param n: int, ngram.
        :return: float, the metric value.
        """
        if len(sentence) == 0:
            return 0.0  # Prevent a zero division
        distinct_ngrams = set(ngrams(sentence, n))
        return len(distinct_ngrams) / len(sentence)


    def distinct_n_corpus_level(sentences, n):
        """
        Compute average distinct-N of a list of sentences (the corpus).
        :param sentences: a list of sentence.
        :param n: int, ngram.
        :return: float, the average value.
        """
        return sum(distinct_n_sentence_level(sentence, n) for sentence in sentences) / len(sentences)

    total_tokens = []
    for i in range(len(list_of_sentences)):
        current_tokens = tokenizer.encode(list_of_sentences[i])  
        total_tokens += current_tokens 

    return distinct_n_sentence_level(total_tokens, n)


def centralized_gradient(x, use_gc=True, gc_conv_only=False):
    '''credit - https://github.com/Yonghongwei/Gradient-Centralization '''
    if use_gc:
        if gc_conv_only:
            if len(list(x.size())) > 3:
                x.add_(-x.mean(dim=tuple(range(1, len(list(x.size())))), keepdim=True))
        else:
            if len(list(x.size())) > 1:
                x.add_(-x.mean(dim=tuple(range(1, len(list(x.size())))), keepdim=True))
    return x



# Setup model, tokenizer, Dataloader

In [10]:
# seed numpy, python random, torch, cuda_torch
pl.seed_everything(params['random_seed'], workers=True)

model = GPT_NEMO(num_intent_classes = 2, num_emotion_classes = 52, hidden_size= params['hidden_size'], img_features_size = params['img_features_size'])

tokenizer = AutoTokenizer.from_pretrained(params['pretrained_model'], 
                        truncation_side='left', )

add_special_tokens(tokenizer, model)

user1_token = tokenizer.additional_special_tokens_ids[0]
user2_token = tokenizer.additional_special_tokens_ids[1]
gen_token = tokenizer.additional_special_tokens_ids[2]
emo_token = tokenizer.additional_special_tokens_ids[3]
img_token = tokenizer.additional_special_tokens_ids[4]
pad_token = tokenizer.pad_token_id
eos_token = tokenizer.eos_token_id
tokenizer.truncation_side = 'left' # phải set lại, tự nhiên nó bị reset thành right ??

#https://discuss.pytorch.org/t/what-is-the-weight-values-mean-in-torch-nn-crossentropyloss/11455/10
weights = [1, 2.33]
class_weights = torch.FloatTensor(weights).to(device)
img_intent_CE = nn.CrossEntropyLoss(weight = class_weights)
emo_CE = nn.CrossEntropyLoss()
huber_loss = nn.HuberLoss()
# huber_loss = nn.MSELoss()

Global seed set to 234


Downloading:   0%|          | 0.00/641 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/335M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  


Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

In [11]:

train_dataset = NemoDataset(data= prefix + config['path']['train_preprocessed_path'])
val_dataset = NemoDataset(data= prefix + config['path']['val_preprocessed_path'])

# Dataloader
train_dataloader = DataLoader(train_dataset, 
                              batch_size=params['batch_size'], 
                               shuffle=True, 
                               num_workers= params['num_cpus'],
                            )

val_dataloader = DataLoader(val_dataset, 
                              batch_size=params['batch_size'], 
#                               shuffle=True
                           )

preprocessing...
preparing inputs...


  0%|          | 0/27725 [00:00<?, ?it/s]

preprocessing...
preparing inputs...


  0%|          | 0/12665 [00:00<?, ?it/s]

# Setup torch lightning, wandb

In [12]:
# Login dô wandb
os.environ["WANDB_API_KEY"] = params['wandb_key']

In [13]:
# 3 cái này sẽ vào list callbacks trong Trainer của torch lightning
#learning rate monitor (log lr mỗi step)
lr_monitor = LearningRateMonitor(logging_interval='step') #log learning rate

# wandb logger, log thông tin lên wandb
wandb_logger = WandbLogger(entity = 'thesis-chatbot',     # Team wandb
                           name = params['wandb_run_name'], # Tên lần chạy (nằm trong project)
                           project = params['wandb_project']) # Tên project

# Early stopping
early_stopping = EarlyStopping(monitor="val/multitask_loss", 
                               mode="min", 
                               min_delta= params['earlystopping_min_delta'], #ngưỡng thay đổi, < thì sẽ count + 1
                               patience= params['earlystopping_patience'], #count = 9 thì ngắt dừng train luôn
                               )

# saves top-K checkpoints based on "val/multitask_loss" metric
checkpoint_callback = ModelCheckpoint(
    save_top_k= 3,
    monitor="val/multitask_loss",
    mode="min",
    dirpath="checkpoints/",
    filename="nemo-epoch={epoch:02d}-val_mtask_loss={val/multitask_loss:.3f}",
    auto_insert_metric_name=False,
    save_weights_only=True,
    every_n_epochs=1,
)


# log gradients and model topology
wandb_logger.watch(model)

# log params
wandb_logger.experiment.config.update(params) # Lưu dictionary params (cell đầu tiên)

# Callback đánh giá (inference sinh câu thật)
class EvaluateCallback(Callback):
    def on_validation_epoch_end(self, trainer, pl_module):
        
        print("Evaluating on train set...")
       
        """
        Chuẩn bị inputs (predicted response), gold_response (actual response)
        Output của hàm này (create_evaluation_inputs) sẽ được đưa vào quá trình đánh giá
        tiến hành đánh giá (bleu, distinct) khi mô hình chạy validate xong
        """
        # lấy thử 60 dòng của tập train ra đánh giá
        evaluation_temp = create_evaluation_inputs(train_dataset, limit_val_set = 60)
        
        list_of_context = evaluation_temp[0]
        list_of_gold_responses = evaluation_temp[1]
        list_of_preds = []
        print("Generating responses for evaluation")
        for context in tqdm(list_of_context):
            pred = generate(context, max_length= 20, top_k= 200, top_p= 0.8, temperature = 0.95)
            # xoá context trong prediction:  ....<speaker1> *response
#             pred = temp.rsplit('>', 1)[1].strip()
            
            list_of_preds.append(pred)
        
        # Log table 30 câu context, resopnse ở trên
        # (context, response, gold_response) dô wandb
        num_rows = 30
        my_dataframe = pd.DataFrame({'Context': list_of_context[:num_rows], 
                        'Response': list_of_preds[:num_rows], 
                        'Gold Response': list_of_gold_responses[:num_rows]})
        
        table = wandb.Table(dataframe=my_dataframe)
        wandb.log({"evaluate on train set": table})
        
        print("Computing metrics")
        bleu_2 = compute_bleu_n(list_of_preds, list_of_gold_responses, n=2)
        bleu_3 = compute_bleu_n(list_of_preds, list_of_gold_responses, n=3)
        bleu_4 = compute_bleu_n(list_of_preds, list_of_gold_responses, n=4)
        
        dist_1 = compute_dist_n(list_of_preds, n=1)
        dist_2 = compute_dist_n(list_of_preds, n=2)
        dist_3 = compute_dist_n(list_of_preds, n=3)
        
        print("train/bleu 2", bleu_2)
        print("train/bleu 3", bleu_3)
        print("train/bleu 4", bleu_4)
        wandb.log({"train/bleu 2": bleu_2})
        wandb.log({"train/bleu 3": bleu_3})
        wandb.log({"train/bleu 4": bleu_4})
        
        print("train/dist_1", dist_1)
        print("train/dist_2", dist_2)
        print("train/dist_3", dist_3)
        wandb.log({"train/dist 1": dist_1})
        wandb.log({"train/dist 2": dist_2})
        wandb.log({"train/dist 3": dist_3})
        
        #---------------------------------------------------
        
        print("Evaluating on val set...")
       
        """
        Chuẩn bị inputs (predicted response), gold_response (actual response)
        Output của hàm này (create_evaluation_inputs) sẽ được đưa vào quá trình đánh giá
        tiến hành đánh giá (bleu, distinct) khi mô hình chạy validate xong
        """
        # Chạy thử limit_val_set (90 dòng) để đánh giá trên tập val
        evaluation_temp = create_evaluation_inputs(val_dataset, limit_val_set = params['limit_val_set'])
        
        list_of_context = evaluation_temp[0]
        list_of_gold_responses = evaluation_temp[1]
        list_of_preds = []
        print("Generating responses for evaluation")
        for context in tqdm(list_of_context):
            pred = generate(context, max_length= 20, top_k= 300, top_p= 0.7, temperature = 1)
            # xoá context trong prediction:  ....<speaker1> *response
#             pred = temp.rsplit('>', 1)[1].strip()
            
            list_of_preds.append(pred)
            
        # Log table 30 câu context, resopnse ở trên
        # (context, response, gold_response) dô wandb
        num_rows = 30
        my_dataframe = pd.DataFrame({'Context': list_of_context[:num_rows], 
                        'Response': list_of_preds[:num_rows], 
                        'Gold Response': list_of_gold_responses[:num_rows]})
        
        table = wandb.Table(dataframe=my_dataframe)
        wandb.log({"evaluate on val set": table})
        
        print("Computing metrics")
        bleu_2 = compute_bleu_n(list_of_preds, list_of_gold_responses, n=2)
        bleu_3 = compute_bleu_n(list_of_preds, list_of_gold_responses, n=3)
        bleu_4 = compute_bleu_n(list_of_preds, list_of_gold_responses, n=4)
        
        dist_1 = compute_dist_n(list_of_preds, n=1)
        dist_2 = compute_dist_n(list_of_preds, n=2)
        dist_3 = compute_dist_n(list_of_preds, n=3)
        
        print("val/bleu 2", bleu_2)
        print("val/bleu 3", bleu_3)
        print("val/bleu 4", bleu_4)
        wandb.log({"val/bleu 2": bleu_2})
        wandb.log({"val/bleu 3": bleu_3})
        wandb.log({"val/bleu 4": bleu_4})
        
        print("val/dist_1", dist_1)
        print("val/dist_2", dist_2)
        print("val/dist_3", dist_3)
        wandb.log({"val/dist 1": dist_1})
        wandb.log({"val/dist 2": dist_2})
        wandb.log({"val/dist 3": dist_3})
        

 # lightning trainer----------------------
trainer = Trainer(
#     deterministic=True, # reproduce kết quả mỗi lần train sẽ giống nhau, cùng với pl.seed_everything ở trên
    logger = wandb_logger, # logger
    default_root_dir="./checkpoints", # checkpoint path
    
#     ckpt_path="some/path/to/my_checkpoint.ckpt", # bật cmt lên nếu muốn resume checkpoint
    
    # list callback hỗ trợ cho trainer 
    callbacks= [lr_monitor,          #log learning rate   
#                 EvaluateCallback(),  #đánh giá metric (bleu, dist)
                # early_stopping,       #dừng sớm
                checkpoint_callback
               ],
    
    max_epochs= params['epochs'], 
    devices="auto", 
    accelerator="auto", # If your machine has GPUs, it will use all the available GPUs for training
    accumulate_grad_batches= params['gradient_accumulation'],
#     progress_bar_refresh_rate = 1,  #tỉ lệ update thanh progress
    log_every_n_steps= 2, # How often to add logging rows
    val_check_interval = params['val_check_interval'],
    limit_val_batches = params['limit_val_set'],
    overfit_batches= params['overfit_batches'], # dùng x batch để ép model overfit (debug)
)

wandb: Currently logged in as: thesis-chatbot (use `wandb login --relogin` to force relogin)


wandb: logging graph, to disable use `wandb.watch(log_graph=False)`
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [14]:
trainer.fit(model, train_dataloader, val_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.

   | Name                     | Type            | Params
--------------------------------------------------------------
0  | gpt                      | GPT2LMHeadModel | 124 M 
1  | img_intent_fc            | Sequential      | 1.5 K 
2  | emo_fc                   | Sequential      | 39.9 K
3  | img_features_fc          | Sequential      | 590 K 
4  | siam_response_gen_img_fc | Sequential      | 590 K 
5  | siam_img_features_fc     | Sequential      | 590 K 
6  | train_intent_accuracy    | Accuracy        | 0     
7  | val_intent_accuracy      | Accuracy        | 0     
8  | train_intent_f1          | F1Score         | 0     
9  | val_intent_f1            | F1Score         | 0     
10 | train_emo_accuracy       | Accuracy        | 0     
11 | val_emo_accuracy         | Accuracy        | 0     
12 | train_emo_f1             | F1Score         | 0     
13 | val_emo_f1              

Sanity Checking: 0it [00:00, ?it/s]

day<|endoftext|><gen> 1 38


Training: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:724: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [15]:
torch.save(model.state_dict(), 'nemo.pt')

In [16]:
wandb.finish()

epoch,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr-AdamW,▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▆▆▆▆▇▇▇█
train/accuracy_emo_step,▁▁▁▁▁▂▁▂▂▂▂▁▄▃▅▃▅▄▅▄██▅▆▆
train/accuracy_img_intent_step,▄█▇▂▄▆▇▂▄▇▇▃▂▇▅▂▃▅▆▁▄▇▆▇█
train/emo_loss_step,█▃▃▂▁▁▂▁▂▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁
train/f1_emo_step,▁▁▁▁▁▂▁▃▂▃▂▂▅▄▆▄▆▄▅▅█▇▆▆▆
train/f1_img_intent_step,▂▇▇▃▄▆▇▄▄██▃▃█▆▃▃▅▆▁▄▇▅▆▇
train/img_intent_loss_step,█▄▂▃▁▂▂▂▃▁▁▂▂▁▁▃▁▂▁▂▁▁▂▁▁
train/lm_loss_step,█▄▃▂▂▂▃▃▄▂▁▂▁▂▁▁▁▂▁▁▁▁▂▁▁
train/multitask_loss_step,█▄▃▂▁▂▂▂▃▁▂▂▂▂▁▂▂▂▂▂▂▁▃▂▁
train/siam_loss_step,█▇▆▆▅▄▄▄▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁


# Load trained model + inference

## Định nghĩa mô hình mới 

In [17]:
# temp = GPT_NEMO()
# temp.gpt.resize_token_embeddings(len(tokenizer))

## Load bằng state_dict (file state_dict lưu bằng torch.save ở trên)

In [18]:
# Load bang native torch
# loadd = torch.load("nemo.pt")
# temp.load_state_dict(loadd)

In [19]:
# Gắn biến toàn cục model thành temp
# model = temp

## Load bằng checkpoint (phải có file checkpoint của torch lightning)

In [20]:
# Load bằng checkpoint
# !ls "checkpoints/GPT_Nemo (multitask, no_img)/3hgawtvl/checkpoints" -l

In [21]:
# checkpoint_path = "checkpoints/GPT_Nemo (multitask, no_img)/1ln5jzrb/checkpoints/epoch=7-step=1735.ckpt"
# temp = GPT_NEMO.load_from_checkpoint(checkpoint_path)

# hong được thì load thử cách này (native torch, ngon luon)
# checkpoint_path = 'checkpoints/nemo-epoch=04-val_mtask_loss=6.011.ckpt'
# loadd = torch.load(checkpoint_path)
# temp.load_state_dict(loadd['state_dict'])

## Inference

In [22]:
# # Inference
# model.to(device)
# context = "<speaker1>Hello<|endoftext|>normal<speaker2>" \
#  "You are a robot<|endoftext|>naughty<speaker1>"\
#  "What is the problem<|endoftext|>normal<speaker2>"\
#  "Is there any problem with my robot IQ<|endoftext|>normal<speaker1>"\
#  "Can I say that my IQ is comparable to yours?<|endoftext|>proud"\
#  "?<|endoftext|>normal<speaker2>"\
#  "Then you say yes<|endoftext|>normal<speaker1>"\
#  "I said you are a robot<|endoftext|>normal<speaker2>"\
#  "What did you say?<|endoftext|>doubt<speaker1>"\
#  "I said I love you<|endoftext|>normal<speaker2>"\
#  "I love you too<|endoftext|>normal<speaker1>"\
#  "You are not human<|endoftext|>normal<speaker2>"\
#  "What is the human<|endoftext|>normal<speaker2>"\
#  "So you are a robot<|endoftext|>naughty<speaker1>"\
#  "Ok<|endoftext|>normal<speaker2>"\
#  "How about you<|endoftext|><speaker1>"\
#  "Do you want me to cry<|endoftext|>normal<speaker2>"\
#  "So I still don’t know what to say<|endoftext|>normal<speaker1>"\
#  "Are you still robot<|endoftext|>normal<speaker2>"\
#  "Ok ok<|endoftext|>normal<speaker1>"

# context = "<speaker1>I'm so sad<|endoftext|>sad<speaker2>"\
# "the tragedy of not loving the girl you like<|endoftext|><normal><speaker1>"\
# "I found it too, my heart is distressed and there is nothing to do<|endoftext|>distressed<speaker2>"\
# "What should I do if I don’t love someone I like<|endoftext|>normal<speaker1>"\
# "Find the right person first, the right person<|endoftext|>normal<speaker2>"\
# "...I can't do it<|endoftext|>excited<speaker1>"\
# "Why?<|endoftext|>normal<speaker2>"\
# "It's just a matter of time<|endoftext|>normal<speaker1>"\
# "It's okay, we are in a different world after all<|endoftext|>normal<speaker2>"\
# "That's right, I understand.<|endoftext|>encourage<speaker1>"\
# "You won't love someone you like in the end.<|endoftext|>normal<speaker2>"\
# "What makes Gemini so attractive.<|endoftext|>normal<speaker1>"\
# "It seems that you are already very good in Gemini.<|endoftext|>normal<speaker2>"\
# "I don't even know you are in love, I just want to be with you forever<|endoftext|>normal<speaker1>"
# "You are so cute<|endoftext|>encourage<speaker2>"\
# "You are my true love!<|endoftext|>encourage<speaker1>"\

context = "<speaker1>Hello<|endoftext|>normal<img><speaker2>"\
"Hello friend, how are you<|endoftext|>normal<img><speaker1>"\
"Good, how are you<|endoftext|>normal<img><speaker2>"\
"Sad, she broke up with me over text<|endoftext|>sad<img><speaker1>"\
"Oh how sad<|endoftext|>sad<img><speaker2>"\
"Should I kill myself ?<|endoftext|>sad<img><speaker1>"\
# "If you want to die, I advise you to take medicine and wait for you to fall in love.<|endoftext|>sad<img><speaker2>"\
# "I am a person who is willing to wait for you<|endoftext|>sad<img><speaker1>"\
# 'Are you willing to wait for me to fall in love with you<|endoftext|>normal<img><speaker2>'\
# 'When will you marry me<|endoftext|>normal<img><speaker1>'\
# "marry you?<|endoftext|>surprise<img><speaker2>"\
# "Yes, yes, yes, marry me<|endoftext|>naughty<img><speaker1>"\
# 'I am so touched by your words<|endoftext|>normal<img><speaker2>'\
# 'I am a person who is willing to wait for you to fall in love with me<|endoftext|normal<img><speaker1>'\
# 'You are my love<|endoftext|>normal<img><speaker2>'\
# 'You are my sunshine<|endoftext|>normal<img><speaker1>'\
# 'Then I will marry you<|endoftext|>normal<img><speaker2>'\
# 'Love you<|endoftext|>normal<img><speaker1>'\
# 'Love you<|endoftext|>normal<img><speaker2>'


model.to(device)
generate(context, max_length = 30 , top_p = 0.8, top_k = 300 , temperature = 0.9)

{'emo': 21,
 'img_intent': 1,
 'response': 'unlaw conduc conclud horizont nodd horizont horizont horizont horizont horizont horizont horizont horizont horizont horizont horizont horizont horizont horizont horizont horizont horizont horizont horizont horizont horizont horizont horizont horizont horizont<gen>'}